In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DESCRIPTION

Stack cloud fraction grids to produce single climatology for 2010.

"""

# Import modules
import numpy as np
import pandas as pd
import glob
import os
import netCDF4
import matplotlib.pyplot as plt

In [25]:
# Define path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

# Define files
file_list = sorted(glob.glob(path + 'modis_cloud_properties/*.nc'))

# Define ice sheet grid
ismip = netCDF4.Dataset(path + 'masks/1km-ISMIP6.nc')
ismip_lon = ismip.variables['lon'][:]
ismip_lat = ismip.variables['lat'][:]

In [26]:
# Define some empty arrays
cloud_fraction = np.zeros(ismip_lat.shape)

for i in range(len(file_list)):
    print('Processing... %.0f out of %.0f' %(i, len(file_list)))

    # Read netCDF
    f = netCDF4.Dataset(file_list[i])

    # Stack
    cloud_fraction = np.dstack((cloud_fraction, f.variables['cloud_fraction'][:]))

# Remove first layer
cloud_fraction = cloud_fraction[:, :, 1:]

# Average
cloud_fraction_mean = np.nanmean(cloud_fraction, axis=2)

Processing... 0 out of 26
Processing... 1 out of 26
Processing... 2 out of 26
Processing... 3 out of 26
Processing... 4 out of 26
Processing... 5 out of 26
Processing... 6 out of 26
Processing... 7 out of 26
Processing... 8 out of 26
Processing... 9 out of 26
Processing... 10 out of 26
Processing... 11 out of 26
Processing... 12 out of 26
Processing... 13 out of 26
Processing... 14 out of 26
Processing... 15 out of 26
Processing... 16 out of 26
Processing... 17 out of 26
Processing... 18 out of 26
Processing... 19 out of 26
Processing... 20 out of 26
Processing... 21 out of 26
Processing... 22 out of 26
Processing... 23 out of 26
Processing... 24 out of 26
Processing... 25 out of 26


/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_12732/3104008009.py:17: RuntimeWarning: Mean of empty slice
  cloud_fraction_mean = np.nanmean(cloud_fraction, axis=2)


In [28]:
###############################################################################
# Save 1 km dataset to NetCDF
###############################################################################
dataset = netCDF4.Dataset(path + 'modis_cloud_properties/cloud_fraction_2010.nc', 
                          'w', format='NETCDF4_CLASSIC')
print('Creating... %s' % path + 'modis_cloud_properties/cloud_fraction_2010.nc')
dataset.Title = "Cloud fraction for 2010 from the MYD06_L2 product"
import time
dataset.History = "Created " + time.ctime(time.time())
dataset.Projection = "WGS 84"
dataset.Reference = "Ryan, J. C., Smith, L. C., et al. (unpublished)"
dataset.Contact = "jryan4@uoregon.edu"

# Create new dimensions
lat_dim = dataset.createDimension('y', ismip_lat.shape[0])
lon_dim = dataset.createDimension('x', ismip_lat.shape[1])

# Define variable types
Y = dataset.createVariable('latitude', np.float32, ('y','x'))
X = dataset.createVariable('longitude', np.float32, ('y','x'))

# Define units
Y.units = "degrees"
X.units = "degrees"

# Create the actual 3D variable
cloud_fraction_nc = dataset.createVariable('cloud_fraction', np.float32, ('y','x'))

# Write data to layers
Y[:] = ismip_lat
X[:] = ismip_lon
cloud_fraction_nc[:] = cloud_fraction_mean

print('Writing data to %s' % path + 'modis_cloud_properties/cloud_fraction_2010.nc')

# Close dataset
dataset.close()

Creating... /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/modis_cloud_properties/cloud_fraction_2010.nc
Writing data to /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/modis_cloud_properties/cloud_fraction_2010.nc
